# Import basic libraries

Let's import some basic stuff

## TODO
- Add the nodes in the CityJSON file, keeping the relationships between nodes and edges (linestrings). The nodes have semantics and atttributes.
- Find a small area and compute widths.
- Create LoD0.1 with single lines and attributes for two-way and same for LoD0.2.

# Ideas for modelling

1. Every city object contains `MultiPoints` (nodes of LoD0.1), `MultiLineStrings` (edges of LoD0.1) and `MultiSurfaces `(for LoD1+).
2. Every city object contains `MultiLineStrings` (geometry of edges of LoD0.1) and `MultiSurfaces` (for LoD1+). The network topology (nodes and edges) is defined in its own `"+network"` portion of the city model.
3. Every city object has its own `MultiSurfaces` (for LoD1+). Then the actual network (nodes, edges and their geometry) is stored in `"+network"`.
4. Every node is its own city object. Every edge is its own city object. Every surface is its own city object. Then `CityObjectGroups` are used to relate them.

### Fix for `fiona`

*`fiona` has an issue with GDAL 3.0. Better set your `GDAL_DATA` path to fiona's installation (contains GDAL 2.4.4) prior to running this script.*

In [1]:
import geopandas
import pandas as pd
import osmnx as ox
import networkx as nx
import shapely

# Load the OSM data

This will load the osm file to a network

In [2]:
# We don't want edges to be loaded twice
ox.config(all_oneway=True)

network_graph = ox.graph_from_file('../data/Breda/Breda extract.osm', simplify=False)
# We simplify the graph afterwards, so that osmids are not "smashed" together
network_graph = ox.simplify_graph(network_graph, strict=False)

We create `UUID` for every edge:

In [3]:
import uuid
import hashlib
import json
from cityhash import CityHash32

hashes = []
for u,v,a in network_graph.edges(data=True):
    temp = {
        "osmid": a['osmid'],
        "length": a['length']
    }
    a['uuid'] = CityHash32(uuid.uuid4().hex)
    hashes.append(a['uuid'])

assert len(hashes) == len(set(hashes))

Then we export the edges to a GeoDataFrame:

In [4]:
nodes, edges = ox.graph_to_gdfs(network_graph)

edges = edges.to_crs("EPSG:28992")

len(edges)

9856

In [5]:
nodes.to_file('output/nodes.geojson', driver='GeoJSON')

## Filter drive and service roads only

Filter roads to only motorways:

In [6]:
highway_types = [
    "primary",
    "secondary",
    "motorway",
    "trunk",
    "tertiary",
    "unclassified",
    "residential",

    "motorway_link",
    "trunk_link",
    "primary_link",
    "secondary_link",
    "tertiary_link",

    "living_street",
    "service",
    "pedestrian",
    "track",
    "bus_guideway",
    "escape",
    "raceway",
    "road"
]

road_edges = edges[(edges['highway'].isin(highway_types)) & (edges['area'] != 'yes')]

road_edges.to_file('output/road_edges.geojson', driver='GeoJSON')
len(road_edges)

685

In [7]:
road_edges.head()

,u,v,key,osmid,area,highway,name,length,uuid,geometry,oneway,maxspeed,landuse,bridge,access,tunnel,service,lanes,ref,width
52,6450151604,43136448,0,7224397,NaN,tertiary,Keizerstraat,10.281,1553522545,"LINESTRING (112852.058 399899.616, 112851.205 ...",yes,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,43155685,4454022829,0,7225340,NaN,service,NaN,8.436,3067046398,"LINESTRING (112517.886 400392.439, 112520.296 ...",NaN,NaN,NaN,yes,no,NaN,NaN,NaN,NaN,NaN
99,1642954984,2197761851,0,7224390,NaN,tertiary,Kloosterplein,5.654,3846808031,"LINESTRING (112993.855 400056.230, 112999.227 ...",yes,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,1642954984,43142237,0,7224394,NaN,tertiary,Vlaszak,8.249,3293179439,"LINESTRING (112993.855 400056.230, 112994.962 ...",yes,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,1401651638,2731824407,0,126447044,NaN,pedestrian,Sint Janstraat,7.449,2159890458,"LINESTRING (112787.440 400066.830, 112783.461 ...",NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN


# Load the BGT data

In [8]:
bgt_roads = geopandas.read_file('../data/Breda/bgt_roads.geojson')

len(bgt_roads)

38859

## Compute primary-secondary road polygons

We need to assign secondary road polygons (parking spaces, sidewalks) to the primary roads

### Filter primary roads

Primary types are those that vehicles are supposed to move:

In [9]:
bgt_road_types = [
    'rijbaan lokale weg',
    'rijbaan regionale weg',
    # 'overweg' # Maybe?
]

bgt_roads =  bgt_roads[bgt_roads['eindRegistratie'].isnull()]

bgt_roads['parent_gml_id'] = bgt_roads['gml_id']

bgt_main_roads = bgt_roads[bgt_roads['function'].isin(bgt_road_types)]

len(bgt_main_roads)

6899

### Identify parent for every secondary road

Find the secondary roads that touch the main ones:

In [10]:
bgt_secondary_roads = bgt_roads[~bgt_roads['function'].isin(bgt_road_types)]

len(bgt_secondary_roads)

19551

Let's overlay the secondary roads with the main roads. The result should be (mostly) linestrings accross their common boundaries:

In [11]:
bgt_common_boundaries = geopandas.overlay(bgt_secondary_roads, bgt_main_roads, how='intersection', keep_geom_type=False)

len(bgt_common_boundaries)

29544

We need to compute the longest common boundary per `gml_id` to pick the respective parent `gml_id`:

In [12]:
bgt_common_boundaries['length'] = bgt_common_boundaries['geometry'].length

max_rows = bgt_common_boundaries[['gml_id_1', 'function_1', 'gml_id_2', 'geometry', 'length']].groupby('gml_id_1')['length'].idxmax()

Now, let's join this with the original secondary rows table to bring back their original geometries:

In [13]:
temp_roads = bgt_common_boundaries.loc[max_rows]
temp_roads = temp_roads[['gml_id_1', 'function_1', 'gml_id_2', 'geometry']]
temp_roads = temp_roads.rename(columns={'gml_id_1': 'gml_id', 'function_1': 'function', 'gml_id_2': 'parent_gml_id'})
temp_roads = temp_roads.set_index('gml_id')

bgt_roads_parented = temp_roads.merge(bgt_secondary_roads.set_index('gml_id'), on='gml_id', suffixes=('', '_right'))
bgt_roads_parented = bgt_roads_parented[['function', 'parent_gml_id', 'geometry_right']].rename(columns={'geometry_right': 'geometry'})
bgt_roads_parented = bgt_roads_parented.reset_index()

bgt_roads_parented = geopandas.GeoDataFrame(bgt_roads_parented, crs='EPSG:28992')

len(bgt_roads_parented)

14235

### Merge primary and secondary roads

Concatenate everything:

In [14]:
bgt_all_roads = pd.concat([bgt_roads_parented, bgt_main_roads[['gml_id', 'function', 'parent_gml_id', 'geometry']]])

bgt_all_roads.to_file('output/bgt_all_roads.geojson', driver='GeoJSON')

len(bgt_all_roads)

21134

# Intersect roads network with polygon
First, we intersect the road lines with the BGT polygons (to create nodes at the polygon boundaries):

In [15]:
roads = geopandas.overlay(road_edges, bgt_all_roads, how='intersection')

Export the intersected roads to `GeoJSON`:

In [16]:
roads.to_file('output/roads.json', driver='GeoJSON')

roads.head()

,u,v,key,osmid,area,highway,name,length,uuid,oneway,...,access,tunnel,service,lanes,ref,width,gml_id,function,parent_gml_id,geometry
0,6450151604,43136448,0,7224397,NaN,tertiary,Keizerstraat,10.281,1553522545,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,b7242fa64-c535-4445-be1c-7bfbe7f9564f,fietspad,bebfd7ae6-7fd0-4d2a-b300-ba3b13525b4d,"LINESTRING (112850.872 399908.820, 112850.743 ..."
1,43136448,43136804,0,7224397,NaN,tertiary,Keizerstraat,13.041,444594187,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,b7242fa64-c535-4445-be1c-7bfbe7f9564f,fietspad,bebfd7ae6-7fd0-4d2a-b300-ba3b13525b4d,"LINESTRING (112850.743 399909.818, 112850.605 ..."
2,6450151604,43136448,0,7224397,NaN,tertiary,Keizerstraat,10.281,1553522545,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,b1f4910c2-00c3-4fce-856a-847a5e337f16,rijbaan lokale weg,b1f4910c2-00c3-4fce-856a-847a5e337f16,"LINESTRING (112852.058 399899.616, 112851.205 ..."
3,43132022,6450151604,0,7224415,NaN,tertiary,Keizerstraat,82.845,2238778749,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,b1f4910c2-00c3-4fce-856a-847a5e337f16,rijbaan lokale weg,b1f4910c2-00c3-4fce-856a-847a5e337f16,"LINESTRING (112857.519 399822.959, 112858.531 ..."
4,43155685,4454022829,0,7225340,NaN,service,NaN,8.436,3067046398,NaN,...,no,NaN,NaN,NaN,NaN,NaN,b7bfd8ed3-cf75-40ac-9af4-7b06596a7879,rijbaan lokale weg,b7bfd8ed3-cf75-40ac-9af4-7b06596a7879,"LINESTRING (112517.886 400392.439, 112520.230 ..."


# Export to CityJSON

Let's export everything to CityJSON.

First, we'll define how the established intersected lines will be translated to `Road` objects:

In [17]:
def process_ring(line, vertices):
    """Returns the boundaries array of a single linear rings.
    
    Arguments:
    line -- The LineString to be processed (represents the ring)
    vertices -- the global vertices list of the CityJSON
    """
    points = [[x, y, 0] for x, y in list(line.coords)]
    indices = [i + len(vertices) for i in range(len(points))]
    for p in points:
        vertices.append(p)

    return indices

def process_linestring(geom, vertices):
    """Returns the boundaries array of a linear element.
    
    This can process LineString or MultiLineString geometries.
    
    Arguments:
    geom -- the (linear) geometry to be processed
    vertices -- the global vertices list of the CityJSON
    """
    if geom.type == "LineString":
        indices = [process_ring(geom, vertices)]
    else:
        indices = []
        
        for l in geom.geoms:
            indices.append(process_ring(l, vertices))
        
    return indices

def create_geometry(geom, lod, vertices):
    """Returns a CityJSON geometry from a single shapely geometry.
    
    This will also append the 'vertices' list with new vertices.
    
    Arguments:
    geom -- the shapely geometry to be processed
    lod -- the lod of the resulting CityJSON geometry
    vertices -- the global vertices list of the CityJSON
    """
    indices = []
    
    if geom.type == "LineString" or geom.type == "MultiLineString":
        geom_type = "MultiLineString"
        
        indices = process_linestring(geom, vertices)
    elif geom.type == "Polygon":
        geom_type = "MultiSurface"
        
        indices = [process_linestring(geom.boundary, vertices)]
    else:
        raise TypeError(geom.type)
    
    return {
        "type": geom_type,
        "lod": lod,
        "boundaries": indices
    }

def create_geom_with_semantics(map_func, features, lod, vertices, geom_column='geometry'):
    """Returns a CityJSON geometry with semantic elements from multiple features.
    
    Semantic elements are semantic lines (for LineString) or surfaces (for
    MultiSurface).
    
    Arguments:
    map_func -- function that maps an input feature to an output semantic element
    features -- list of features to be processed
    lod -- the lod of the resulting CityJSON geometry
    vertices -- the global vertices list of the CityJSON
    """
    boundaries = []
    semantics = {
        "surfaces": [],
        "values": []
    }
    
    # Get the resulting geometry type from the first feature
    geom = features[0][geom_column]
    if geom.type == "LineString" or geom.type == "MultiLineString":
        geom_type = "MultiLineString"
    elif geom.type == "Polygon":
        geom_type = "MultiSurface"
    else:
        raise TypeError(geom.type)
    
    # Process all features
    i = 0
    for f in features:
        surface = map_func(f)
        
        if geom_type == "MultiLineString":
            indices = process_linestring(f[geom_column], vertices)
            for l in indices:
                boundaries.append(l)
                semantics["surfaces"].append(surface)
                semantics["values"].append(i)
                i = i + 1
        else:
            indices = process_linestring(f[geom_column].boundary, vertices)
        
            boundaries.append(indices)
            semantics["surfaces"].append(surface)
            semantics["values"].append(i)
            i = i + 1
    
    return {
        "type": geom_type,
        "lod": lod,
        "boundaries": boundaries,
        "semantics": semantics
    }

def osm_semantics_map(feature):
    obj = {
        "type": feature['highway'],
        "osm_id": feature['osmid'],
        "oneway": feature['oneway'] if isinstance(feature['oneway'], str) else "no"
    }
    
    if isinstance(feature['name'], str):
        obj["name"] = feature['name']
    
    if isinstance(feature['maxspeed'], str):
        obj["maxspeed"] = feature['maxspeed']
    
    return obj

def bgt_semantics_map(feature):
    return {
        "type": feature['function'],
        "gml_id": feature['gml_id']
    }

def create_cityobject(feature, vertices):
    """Create a CityJSON city object from a single OSM linear features."""
    return {
        "type": "Road",
        "attributes": {
            "osm_id": feature['osmid'],
            "highway": feature['highway'],
        },
        "geometry": [
            create_geometry(feature['geometry'], "0.1", vertices)
        ]
    }

Now, let's run this against all intersected road segments:

In [18]:
vertices = []
objects = {}

road_groups = roads.groupby('parent_gml_id')

for gml_id, group in road_groups:
    features = [f for i, f in group.iterrows()]
    objects[gml_id] = {
        "type": "Road",
        "geometry": [
            create_geom_with_semantics(osm_semantics_map,
                                       features,
                                       '0.1',
                                       vertices)
        ]
    }

bgt_roads_groups = bgt_all_roads.groupby('parent_gml_id')

for main_gml_id, group in bgt_roads_groups:
    if main_gml_id in objects:
        features = [f for i, f in group.iterrows()]
        objects[main_gml_id]['geometry'].append(
            create_geom_with_semantics(bgt_semantics_map,
                                       features,
                                       '2',
                                       vertices)
        )

Finally, let's export everything as CityJSON:

In [19]:
import json
import io

output = {
  "type": "CityJSON",
  "version": "1.0",
  "CityObjects": objects,
  "vertices": vertices
}

with open('output/breda.json', 'w') as file:
    json.dump(output, file)

## Fix missing network and polygons
We should create individual road objects for the polygons (and respective network segments) that do not have a `parent_gml_id`.

# Calculate carriageways and lanes
- Compute the carriageways only from OSM attributes.
- Compute road width:
     - We exclude intersections.
- How we create the carriageways for twoway streets and more specifically for intersections.

NOTE: Case of a two-way road where we figure out that the road width doesn't fit two lanes (special case were two ways share the same carriageway).

In [20]:
from shapely.ops import unary_union
from shapely.geometry import Point, LineString, MultiLineString

def get_vertex(line, first=True):
    """Returns the first or last vertex"""
    if line.type == "LineString":
#         print(len(line.boundary))
        return line.boundary[0 if first else len(line.boundary) - 1]
    elif line.type == "MultiLineString" and len(line.geoms) == 1:
        return line.geoms[0].boundary[0 if first else len(line.boundary) - 1]
    else:
        raise TypeError("This is MultiLineString with many parts!")

## Calculate width

### Using PostGIS

This is to calculate the widths in PostGIS using [this](https://github.com/willemhoffmans/bgt_wegbreedte/).

Let's import stuff and create the connection with the database:

In [21]:
# SO https://gis.stackexchange.com/a/239231

# Imports
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import pandas as pd
import geopandas as gpd

pg_user = "roads"
pg_pass = "roads"
pg_host = "localhost"
pg_port = 5432
pg_database = "road_widths"

# Creating SQLAlchemy's engine to use
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(pg_user, pg_pass, pg_host, pg_port, pg_database))

Now we export the BGT roads to the database with the appropriate names:

In [22]:
export_bgt = bgt_roads.copy()
export_bgt['geometrie_vlak'] = export_bgt['geometry'].apply(lambda x: WKTElement(x.wkt, srid=28992))

export_bgt.drop('geometry', 1, inplace=True)

export_bgt = export_bgt.rename(columns={'function':'bgt_functie'})

# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
export_bgt.to_sql("wegdeel", engine, schema='bgt', if_exists='replace', index=False, 
                         dtype={'geometrie_vlak': Geometry('POLYGON', srid=28992)})

Let's create the required functions in the database:

In [23]:
import psycopg2

conn = psycopg2.connect(dbname=pg_database, user=pg_user, password=pg_pass)

file = open("wegbreedtes_script.sql", "r")
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
cursor = conn.cursor()
cursor.execute(file.read())

Export the roads:

In [24]:
from shapely.geometry import MultiLineString

export_roads = road_edges.copy()
export_roads['geom'] = export_roads['geometry'].apply(lambda x: WKTElement(MultiLineString([x]).wkt, srid=28992))

export_roads.drop('geometry', 1, inplace=True)

# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
export_roads.to_sql("road_edges", engine, if_exists='replace', index=False, 
                         dtype={'geom': Geometry('MULTILINESTRING', srid=28992)})

Now, run the function to compute the widths:

In [25]:
conn = psycopg2.connect(dbname=pg_database, user=pg_user, password=pg_pass)
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)

cursor = conn.cursor()

cursor.execute("SELECT wh_wegbreedte_bgt_generiek ('road_edges', 'uuid', false, 0);")
for c in cursor:
    print(c)

('OK!',)


Load the resulting table with widths:

In [26]:
roads_with_width = geopandas.GeoDataFrame.from_postgis("SELECT * FROM breedte_analyse_nieuw.gemiddeldebreedte", conn, geom_col='geom' )

roads_with_width = roads_with_width.rename(columns={"geom": "geometry"})

roads_with_width.head()

,gid,typeweg,wegvakid,geometry,gem_breedte,dev_breedte,min_breedte,max_breedte,oper_breedte,afst_s,aantal,aantal_op
0,115,rijbaan lokale weg,790112628,"MULTILINESTRING ((112015.215 399976.240, 11205...",3.08,0.37,2.93,4.06,2.96,0.0,9,8
1,1,rijbaan lokale weg,10062787,"MULTILINESTRING ((112432.605 400094.208, 11243...",3.60,0.01,3.60,3.62,3.60,0.0,9,8
2,2,rijbaan lokale weg,10335401,"MULTILINESTRING ((112500.255 400501.820, 11253...",8.01,4.59,5.32,19.32,6.60,0.0,9,8
3,3,rijbaan lokale weg,15515759,"MULTILINESTRING ((112194.202 400345.174, 11220...",8.42,2.93,6.21,13.69,7.16,0.0,9,7
4,4,rijbaan lokale weg,19944829,"MULTILINESTRING ((112491.728 400527.837, 11248...",10.44,4.70,6.07,19.41,9.32,0.0,9,8


# Create carriageways

We calculated the widths via PostGIS using [this](https://github.com/willemhoffmans/bgt_wegbreedte/) (embarassingly, the previous scripts didn't work).

Let's load the data with the width:

In [540]:
roads_width_final_final = roads_with_width.rename(columns={"wegvakid": "uuid"}).merge(road_edges.set_index("uuid"), on="uuid", suffixes=('', '_right'))

len(roads_width_final_final)

608

Append the missing roads segments. We will set a road width of 0 for them. Also, we need to convert them to `MultiLineString` to match the rest of the geometries.

In [571]:
from shapely.geometry import MultiLineString

def make_multilinestring(f):
    """Returns a geometry as MultiLineString if it is a LineString"""
    
    if f["geometry"].type == "LineString":
        return MultiLineString([f["geometry"]])
    
    return f["geometry"]

roads_width_final_final = pd.concat([roads_width_final_final, road_edges[road_edges['uuid'].isin(roads_width_final_final['uuid']) == False]], ignore_index=True)

roads_width_final_final.loc[roads_width_final_final[np.isnan(roads_width_final_final['oper_breedte'])].index, 'oper_breedte'] = 0

roads_width_final_final["geometry"] = roads_width_final_final.apply(make_multilinestring, axis=1)

len(roads_width_final_final)

685

Let's create the basic carriageways when `is_one_carriageway` is `false` (specific width and a two way street):

In [572]:
width_col = 'oper_breedte'

def find_neighbouring_edges(f, nodeid):
    """Returns the road UUIDs of the neighbours of a feature at a given node"""
    
    uuids = [a['uuid'] for u, v, a in network_graph.in_edges(nodeid, data=True)] + [a['uuid'] for u, v, a in network_graph.out_edges(nodeid, data=True)]
    
    uuids.remove(f['uuid'])
    
    return roads_width_final_final[roads_width_final_final['uuid'].isin(uuids)]

def is_one_carriageway(f, tolerance):
    return (f[width_col] < (6 - tolerance)) or (f[width_col] > (15 + tolerance))

def carriageway_placement(f):
    """Returns the width of the carriage ways"""
    if f['oneway'] == 'yes':
        return 0
    
    u_neighbours = find_neighbouring_edges(f, f['u'])
    v_neighbours = find_neighbouring_edges(f, f['v'])
    
    if len(u_neighbours) == 1 and len(v_neighbours) == 1:
        if not is_one_carriageway(u_neighbours.iloc[0], 0) and not is_one_carriageway(v_neighbours.iloc[0], 0):
            return f[width_col] / 4
    
    # Check if this is the last segment of a road which normally, if short, is
    # very much affected by the intersection and will end up with an unrealistically
    # high width. Therefore, we will choose its type by the previous segment of this
    # road.
    if f['length'] < 30:
        if len(u_neighbours) > 1 and len(v_neighbours) == 1:
            n = v_neighbours.iloc[0]
            if is_one_carriageway(n, 0):
                return 0
            else:
                return n[width_col] / 4
        if len(u_neighbours) == 1 and len(v_neighbours) > 1:
            n = u_neighbours.iloc[0]
            if is_one_carriageway(n, 0):
                return 0
            else:
                return n[width_col] / 4
    
    # This is only the general case
    if is_one_carriageway(f, 0):
        return 0
    else:
        return f[width_col] / 4

def create_carriageway(f):
    geom = f['geometry']
    
    width = carriageway_placement(f)
    if width == 0:
        return geom
    
    # TODO: We check if it's an extreme and then we check if it's a dead-end
    #       in which case we just exclude or we return the original geometry
    
    return geom.geoms[0].parallel_offset(width, 'left').union(geom.geoms[0].parallel_offset(width, 'right'))

# geopandas.GeoSeries(roads_width_final_final.apply(create_carriageway, axis=1)).plot()

In [573]:
roads_width_final_final['carriage_ways'] = roads_width_final_final.apply(create_carriageway, axis=1)

geopandas.GeoDataFrame(roads_width_final_final[['u', 'v', 'carriage_ways']], geometry='carriage_ways', crs='EPSG:28992').to_file('output/ok.geojson', driver='GeoJSON')

## Join carriageways

Now, we join the individual network segments with each other:
1. if both are single carriageway there is nothing to do
2. if both are dual carriageways then we compute the midpoint of the end and start node and we join them there
3. if one is single and another is dual then we join the dual to the end node of the single one (making a triangle).

We also identify continuation between carriageways in intersections and join them.

In [574]:
from shapely.ops import substring, linemerge
from shapely.geometry import LineString, MultiLineString
import numpy as np

def is_first_vertex(f, u_nodeid):
    return f['u'] == u_nodeid

def cut_line(geom, dist, reverse=False):
    if reverse:
        return substring(geom, 0, geom.length - dist)
    else:
        return substring(geom, dist, geom.length)

def reverse_line(line):
    """Returns the line in the reverse orientation"""
    return LineString(reversed(line.coords))

def midpoint(pa, pb):
    return Point((pa.x + pb.x) / 2, (pa.y + pb.y) / 2)

def azimuth(point1, point2):
    '''azimuth between 2 shapely points (interval 0 - 360)'''
    angle = np.arctan2(point2.x - point1.x, point2.y - point1.y)
    return np.degrees(angle) if angle >= 0 else np.degrees(angle) + 360

def line_azimuth(line, beginning, min_length=0.001):
    """Return the azimuth of a LineString.
    
    Depending on the boolean provided, it will return the azimuth of the
    beginning (if True) or end (if False) of the LineString. In order to
    ensure stability, the algorithm expects that a min_length of segments
    is accounted for the two points tha will compute the azimuth.
    """
    
    if line.type == "MultiLineString":
        l = linemerge(line)
    else:
        l = line
    
    start_p = Point(l.coords[0 if beginning else -1])
    c_count = len(l.coords)
    for i in range(1, c_count):
        end_p = Point(l.coords[i if beginning else c_count - i - 1])
        tmp_line = LineString([start_p, end_p])
        if tmp_line.length > min_length:
            return azimuth(start_p, end_p)
    
    # If the min_length is too big, return the azimuth of the whole thing
    return azimuth(start_p, end_p)

def count_carriageways(f):
    if f['carriage_ways'].type == "LineString":
        return 1
    else:
        return len(f['carriage_ways'].geoms)

def as_line(geom):
    """Return a LineString from geom regardless if it's a MultiLineString"""

    if geom.type == "LineString":
        return geom
    else:
        return geom.geoms[0]

def merge_dual_to_single(geom, p, firstVertex):
    left_cw = geom.geoms[0] # Get left carriage way
    right_cw = geom.geoms[1] # Get right carriage way
    
    if left_cw.length < 1 or right_cw.length < 1:
        left_p = get_vertex(left_cw, not firstVertex) # Last point of left line
        right_p = get_vertex(right_cw, firstVertex) # First point of right line

        left_line = LineString([p, left_p])
        right_line = LineString([right_p, p])
    else:
        left_line = cut_line(left_cw, 1, not firstVertex)
        right_line = cut_line(right_cw, 1, firstVertex)

        left_p = get_vertex(left_line, firstVertex)
        right_p = get_vertex(right_line, not firstVertex)

        # Create the left and right side of the "triangle"
        left_side = LineString((p, left_p))
        right_side = LineString((p, right_p))

        # Merge the parts together
        left_line = linemerge(left_side.union(left_line))
        right_line = linemerge(right_side.union(right_line))

        if not get_vertex(left_line, firstVertex).equals(p):
            left_line = reverse_line(left_line)
        if not get_vertex(right_line, not firstVertex).equals(p):
            right_line = reverse_line(right_line)

    return MultiLineString([left_line, right_line])

def join_dual_ways(geom, other_geom, isU, firstVertex):
    other_left_p = get_vertex(other_geom.geoms[0], isU)
    other_right_p = get_vertex(other_geom.geoms[1], not isU)

    if isU == firstVertex:
        temp_p = other_left_p
        other_left_p = other_right_p
        other_right_p = temp_p

    left_cw = geom.geoms[0] # Get left carriage way
    right_cw = geom.geoms[1] # Get right carriage way
    
    other_left_cw = other_geom.geoms[1 if isU == firstVertex else 0]
    other_right_cw = other_geom.geoms[0 if isU == firstVertex else 1]
    
    theta = line_azimuth(left_cw, firstVertex, min_length=5)
    theta = theta - line_azimuth(other_left_cw, isU, min_length=5)
    theta = abs(theta)
    
    straight = 135 < theta < 225 or 315 < theta or theta < 45

    p = left_cw.intersection(other_left_cw)
    if p.type == "Point" and not p.is_empty:
        proj = left_cw.project(p)
        if firstVertex:
            left_line = substring(left_cw, proj, left_cw.length)
        else:
            left_line = substring(left_cw, 0, proj)
    else:
        left_p = get_vertex(left_cw, firstVertex) # First point of left line
        new_left_p = midpoint(left_p, other_left_p)
        
        if firstVertex:
            left_line = LineString([new_left_p] + left_cw.coords[1 if straight else 0:])
        else:
            left_line = LineString(left_cw.coords[:-1 if straight else None] + [new_left_p])
    
    p = right_cw.intersection(other_right_cw)
    if p.type == "Point" and not p.is_empty:
        proj = right_cw.project(p)
        if firstVertex:
            right_line = substring(right_cw, 0, proj)
        else:
            right_line = substring(right_cw, proj, right_cw.length)
    else:
        right_p = get_vertex(right_cw, not firstVertex) # Last point of right line
        new_right_p = midpoint(right_p, other_right_p)
        if firstVertex:
            right_line = LineString(right_cw.coords[:-1] + [new_right_p])
        else:
            right_line = LineString([new_right_p] + right_cw.coords[1:])

    return MultiLineString([left_line, right_line])

def get_line_azimuth(line, reverse):
    """Returns the azimuth of a line based on its boundaries (end points).
    
    Better use the `line_azimuth` function instead!
    """
    if reverse:
        return azimuth(Point(line.coords[-1]), Point(line.coords[-2]))
    
    return azimuth(Point(line.coords[0]), Point(line.coords[1]))

def find_next_road(f, neighbours, nodeid, firstVertex):
    """Returns the feature that counts as a continuation of this road (if any)"""
    
    if len(neighbours) == 0:
        return None
    
    if len(neighbours) == 1:
        return neighbours.iloc[0]
    
    other_feature = []
    
    axis_geom = as_line(f['geometry'])
    a = get_line_azimuth(axis_geom, not firstVertex)
    
    # Find out if there is only one other road with the same number of carriageways
    similar = 0 # number of similar carriageway-type roads in this intersection
    cw_count = count_carriageways(f)
    for i in range(len(neighbours)):
        n = neighbours.iloc[i]
        if count_carriageways(n) == cw_count:
            similar = similar + 1
            other_feature = n
    
    ccc = neighbours.apply(count_carriageways, axis=1) == (1 if cw_count == 2 else 2)
    force_same_cw = len(neighbours[ccc]) == 2
    
    if similar != 1:
        # If more than one similar roads are adjacent, pick the neighbour with the closest to 180° angle
        other_feature = []
        for i in range(len(neighbours)):
            n = neighbours.iloc[i]
            n_a = get_line_azimuth(as_line(n['geometry']), n['u'] != nodeid)
            if abs(n_a - a) < 200 and abs(n_a - a) > 160:
                if cw_count == 1 and force_same_cw and count_carriageways(n) != cw_count:
                    continue
                other_feature = n
    
    if isinstance(other_feature, list):
        return None
    else:
        return other_feature

def join_side(geom, nodeid, f, neighbours, next_feature, firstVertex=True):
    """Returns a geometry that has joined with the adjacent geometry of one side.
    
    geom --- The geometry to alter.
    nodeid --- The node id around which the join will occur.
    f --- The input feature.
    neighbours --- The list of neighbouring features in this intersection
    next_feature --- The feature classified as a "continuation" of this one
    firstVertex --- True if the first vertex of the geometry will be altered, or False if it's the last one.
    """
    if len(neighbours) == 0:
        return geom
    
    if all(count_carriageways(neighbours.iloc[i]) == 1 for i in range(len(neighbours))):
        # Pick first feature, because they are all single carriageways and they meet at the same point
        other_feature = neighbours.iloc[0]
        
        p = get_vertex(other_feature['geometry'], other_feature['u'] == nodeid)

        return merge_dual_to_single(geom, p, firstVertex)
    
    if not next_feature is None:
        if count_carriageways(next_feature) == 1:
            p = get_vertex(next_feature['geometry'], next_feature['u'] == nodeid)
            return merge_dual_to_single(geom, p, firstVertex)
        else:
            return join_dual_ways(geom, next_feature['carriage_ways'], next_feature['u'] == nodeid, firstVertex)
    
    return geom

def get_uuid(f):
    """Returns the uuid of a feature or np.nan"""
    
    if f is None:
        return np.nan
    else:
        return f['uuid']

def join_carriageways(f):
    """Returns the carriageways processed so that they are joined with their neighbours"""
    
    # Identify degenerate case where this is a short segment (<1m) and
    # is adjacent to single cariageways, so we collapse it.
    u_neighbours = find_neighbouring_edges(f, f['u'])
    v_neighbours = find_neighbouring_edges(f, f['v'])
    
    u_next = find_next_road(f, u_neighbours, f['u'], True)
    v_next = find_next_road(f, v_neighbours, f['v'], False)
    
    if count_carriageways(f) == 1:
        return [f['carriage_ways'], get_uuid(u_next), get_uuid(v_next)]

    
    if f['length'] < 1:
        if len(u_neighbours) == 0 or len(v_neighbours) == 0:
            return [f['geometry'], np.nan, np.nan]
        
        if len(u_neighbours) == 1 and len(v_neighbours) == 1 and count_carriageways(u_neighbours.iloc[0]) == 1 and count_carriageways(v_neighbours.iloc[0]) == 1:
            return [f['geometry'], get_uuid(u_next), get_uuid(v_next)]
    
    geom = join_side(f['carriage_ways'], f['u'], f, u_neighbours, u_next, True)
    geom = join_side(geom, f['v'], f, v_neighbours, v_next, False)
    
    return [geom, get_uuid(u_next), get_uuid(v_next)]

roads_width_final_final['fixed_carriageways'], roads_width_final_final['u_next'], roads_width_final_final['v_next'] = zip(*roads_width_final_final.apply(join_carriageways, axis=1))

geopandas.GeoDataFrame(roads_width_final_final[['uuid', 'u', 'v', 'u_next', 'v_next', 'osmid', 'name', 'oneway', 'fixed_carriageways']], geometry='fixed_carriageways', crs='EPSG:28992').to_file('output/fixed_carriageways.geojson', driver='GeoJSON')

**WARNING:** 2495911722 should have a `v_next` because it's opposite road assigned it as it's next feature.

## Split carriageways to features

Carriageways should be now converted to individual features:

In [667]:
from shapely.geometry import LineString
import uuid
from cityhash import CityHash32

def create_uuid(f):
    return CityHash32(uuid.uuid4().hex)

def reverse_if_dual(f):
    if f['dual']:
        return LineString(f['geom'].coords[::-1])
    
    return f['geom']

carriageways = roads_width_final_final.copy().rename(columns={
                                                             'geometry': 'axis_geom',
                                                             'fixed_carriageways': 'geom',
                                                             'uuid': 'road_uuid'
                                                             })

carriageways.drop('carriage_ways', 1, inplace=True)
carriageways.drop('gid', 1, inplace=True)

carriageways = geopandas.GeoDataFrame(carriageways, geometry="geom", crs="EPSG:28992")

# Compute "dual" carriageway attribute
carriageways['dual'] = carriageways.apply(lambda f: len(f['geom'].geoms) > 1, axis=1)

# Explode the MultiLineStrings to LineStrings (so duals will become two features)
carriageways = carriageways.explode()

# Make the exploding index a value to define the side of the carriageway: 0 - left, 1 - right
carriageways = carriageways.reset_index(level=-1).rename(columns={'level_1': 'side'})

# Reverse geometries of ex-dual-carriageways
carriageways['geom'] = carriageways.apply(reverse_if_dual, axis=1)

# Create UUIDs
carriageways['uuid'] = carriageways.apply(create_uuid, axis=1)

# Make sure no collisions exist between UUIDs
assert len(pd.unique(carriageways['uuid'])) == len(carriageways['uuid'])

carriageways.drop(['axis_geom', 'geometry_right'], 1).to_file('output/carriageways.geojson', driver='GeoJSON')

In [576]:
pd.unique(geopandas.GeoSeries(carriageways['geom']).type)

array(['LineString'], dtype=object)

## Fix intersections

Given the continuous carriageways, we need to deal with intersections now.

### Trim and split lines at intersections

We need to make sure that lines are splitted (if they "continue" across the interesction) or get trimmed (if that's their end). We determine contiuation or termination by the existence or absense (respectively) of a `u_next`/`v_next` value.

At the end of this step, we have all carriageways as individual `MultiLineStrings`, which contain one or more `LineStrings` that are topologically valid. So, later on we'll have to explode those `MultiLineStrings` to get the individual carriageway segments.

In [668]:
from shapely.ops import substring, split

def get_feature(df, col_name, value):
    return df[df[col_name] == value].iloc[0]

def find_neighbouring_features(G, nodeid, df, uuid_col='uuid', exclude=[], geom_col='geom', min_length=3.5):
    """Returns the road UUIDs of the neighbours of a feature at a given node
    
    G --- The network to lookup
    nodeid --- The id of the node for which to look for neighbours
    df --- The dataframe to find features
    uuid_col --- The name of the column that contains the edge's uuid
    exclude --- A list of uuids to exclude from the returning list
    geom_col --- The geometry column to check for min_length
    min_length --- If a road segment is shorter than this, then also "bring" the next one
    """
    
    uuids = [a['uuid'] for u, v, a in G.in_edges(nodeid, data=True)] + [a['uuid'] for u, v, a in G.out_edges(nodeid, data=True)]
    
    if isinstance(exclude, list):
        for i in exclude:
            if i in uuids:
                uuids.remove(i)
    else:
        if exclude in uuids:
            uuids.remove(exclude)
    
    new_uuids = []
    for uuid in uuids:
        res = df[df[uuid_col] == uuid]
        if len(res) > 0:
            f = res.iloc[0]
            if f[geom_col].length < min_length:
                if not np.isnan(f['u_next']):
                    new_uuids.append(int(f['u_next']))
                if not np.isnan(f['v_next']):
                    new_uuids.append(int(f['v_next']))
    
    uuids = uuids + new_uuids
    
    if isinstance(exclude, list):
        for i in exclude:
            if i in uuids:
                uuids.remove(i)
    else:
        if exclude in uuids:
            uuids.remove(exclude)
    
    return df[df[uuid_col].isin(uuids)]

def move_end_vertex(line, p, firstVertex):
    """Move the first or last vertex of a line to the given point (p)."""
    
    if firstVertex:
        coords = [p] + line.coords[1:]
    else:
        coords = line.coords[:-1] + [p]
    
    return LineString(coords)

def is_dual_merge(lines, nodeid):
    """Returns True if the two carriageways of road_uuid meet at nodeid
    
    lines --- A dataframe that should contain both carriageways
    nodeid --- The node to check against
    """
    
    left = lines[lines['side'] == 0].iloc[0] # TODO: Add an assert here
    right = lines[lines['side'] == 1].iloc[0] # TODO: Add an assert here
    
    p_left = left['geom'].boundary[1 if left['u'] == nodeid else 0]
    p_right = right['geom'].boundary[0 if right['u'] == nodeid else 1]
    
    return p_left.equals(p_right)
    

def trim_at_intersection(geom, f, nodeid, force_split=False):
    neighbours = find_neighbouring_features(network_graph,
                                            nodeid,
                                            df=carriageways,
                                            uuid_col='road_uuid',
                                            exclude=f['road_uuid'])
    
    # This marks the special case where this segment is dealt as the "not last"
    # because the next one is too short and we are dealing with it.
    not_last_segment = False
    
    # This is to bypass a special case where the next segment is too small and this segment
    # might eventually be crossing with the next segment's perpendicular dual carriageway road.
    if f["dual"] == False:
        next_id = f["u_next" if f['u'] == nodeid else "v_next"]
        if not np.isnan(next_id):
            next_road = carriageways[carriageways['road_uuid'] == int(next_id)].iloc[0]
            if next_road['geom'].length < 3:
                next_neighbours = find_neighbouring_features(network_graph,
                                                        next_road['v' if nodeid == next_road['u'] else 'u'],
                                                        df=carriageways,
                                                        uuid_col='road_uuid',
                                                        exclude=[f['road_uuid'], next_road['road_uuid']])
                neighbours = pd.concat([neighbours, next_neighbours])
                                                
                force_split = not np.isnan(next_road["v_next" if next_road['u'] == nodeid else "u_next"])
                
                not_last_segment = True
    
    # Find the crossing point
    all_lines = shapely.ops.unary_union(neighbours.reset_index()['geom'])
    cross_p = all_lines.intersection(geom)
    
    if cross_p.is_empty:
        if geom.type == "LineString":
            return MultiLineString([geom])
        else:
            return geom
    
    # Split the line according to the other lines
    parts = list(split(geom, all_lines).geoms)
    
    for p in parts:
        if p.length < 0.001:
            parts.remove(p)
    
    # Case where we just split
    if len(parts) == 1:
        return MultiLineString(parts)
    
    if f['dual'] == False:
        if f['u'] == nodeid:
            end_p = f['geom'].boundary[0]
        else:
            end_p = f['geom'].boundary[1]
        
        dest_p = None
        is_merge = False
        for i, n in neighbours.set_index('uuid').iterrows():
            if n['dual'] == True and n['geom'].intersects(geom):
                pp = n['geom'].intersection(geom)
                pp = pp.difference(geom.boundary)
                # TODO: We need to exclude cases where this dual carriageway meets its other part on this side
                if pp.is_empty:
                    continue
                
                is_start = (n['u'] == nodeid) == (n['side'] == 1)

                # Check if this is a merging dual carriageway
                if is_dual_merge(neighbours[neighbours['road_uuid'] == n['road_uuid']], nodeid):
                    dest_p = pp
                else:
                    dest_p = n['geom'].boundary[0 if is_start else 1]
                
                break

        if cross_p.type == "MultiPoint" and not_last_segment == False:
            if is_merge:
                if f['u'] == nodeid:
                    parts = parts[1:]
                else:
                    parts = parts[:-1]
            else:
                if f['u'] == nodeid:
                    first_part = parts[0]
                    first_part = move_end_vertex(first_part, dest_p, False)

                    second_part = parts[1]
                    second_part = move_end_vertex(second_part, dest_p, True)

                    rest = parts[2:]

                    parts = [first_part, second_part] + rest
                else:
                    main_part = parts[:-2]

                    second_to_last_part = parts[-2]
                    second_to_last_part = move_end_vertex(second_to_last_part, dest_p, False)

                    last_part = parts[-1]
                    last_part = move_end_vertex(last_part, dest_p, True)

                    parts = main_part + [second_to_last_part, last_part]

    # Cases where we might split and move the vertices or create a twoway segment
    if force_split:
        # We need to check some things for single carriageways
        next_uuid = f['u_next'] if f['u'] == nodeid else f['v_next']
        # Special case where we need to duplicate the last part of the linestring (to make a twoway)
        if f['oneway'] == "yes" and len(neighbours.groupby('road_uuid')) == 3:            
            next_f = neighbours[neighbours['road_uuid'] == next_uuid].iloc[0]
            if next_f['dual'] == True:
                if f['u'] == nodeid:
                    parts = [reverse_line(parts[0])] + parts
                else:
                    parts = parts + [reverse_line(parts[-1])]
        
        # Special TriAn(na)gle
        next_segment = carriageways[carriageways['road_uuid'] == next_uuid].iloc[0]
        if cross_p.type == "MultiPoint" and f['dual'] == True and next_segment["dual"] == False:
            if (f['u'] == nodeid) == (f['side'] == 0):
                return MultiLineString(parts[:-1])
            else:
                return MultiLineString(parts[1:])
        
        return MultiLineString(parts)
    
    if cross_p.type == "MultiPoint" and (f['geom'].boundary[0] in cross_p.geoms or f['geom'].boundary[1] in cross_p.geoms) and not not_last_segment:
        return MultiLineString(parts)
    
    if cross_p in f['geom'].boundary:
        return MultiLineString(parts)
    
    if f['u'] == nodeid:
        if f['dual'] == True and f['side'] == 0:
            return MultiLineString(parts[:-1])
        else:
            return MultiLineString(parts[1:])
    else:
        if f['dual'] == True and f['side'] == 0:
            return MultiLineString(parts[1:])
        else:
            return MultiLineString(parts[:-1])

def trim_feature(f):
    geom = trim_at_intersection(f['geom'], f, f['u'], not np.isnan(f['u_next']))
    return trim_at_intersection(geom, f, f['v'], not np.isnan(f['v_next']))

carriageways['trimmed'] = carriageways.apply(trim_feature, axis=1)

geopandas.GeoDataFrame(carriageways.drop(['axis_geom', 'geometry_right', 'geom'], 1),
                       geometry="trimmed",
                       crs="EPSG:28992").to_file('output/carriageways_trimmed.geojson', driver='GeoJSON')

# f = carriageways[carriageways['uuid'] == 3457732787].iloc[0]
# trim_feature(f)

In [591]:
pd.unique(geopandas.GeoSeries(carriageways['trimmed']).type)

array(['MultiLineString'], dtype=object)

### Extend and fix complex intersections

If a carriageway meets a dual road that is continuous, then we need to do certain things:
- In case of no continuation of the carriageway, we need to extend to the end node of the opposite side
- In case of continuation where this is dual and the next road segment is single, we need to make a triangle.

**TODO:** Deal with degenerate case of sideroads. Definition: an intersection of 4 roads, with two being dual and two being single. If the dual are a continuation of ~180° and the two single are <60° we have to move them etc.

In [713]:
def extend_at_intersection(geom, f, nodeid):
    neighbours = find_neighbouring_features(network_graph,
                                            nodeid,
                                            df=carriageways,
                                            uuid_col='road_uuid',
                                            exclude=f['road_uuid'])

    new_lines = list(geom.geoms)
    
    # This is to bypass a special case where the next segment is too small and this segment
    # might eventually be crossing with the next segment's perpendicular dual carriageway road.
    if f["dual"] == False:
        next_id = f["u_next" if f['u'] == nodeid else "v_next"]
        if not np.isnan(next_id):
            next_road = carriageways[carriageways['road_uuid'] == int(next_id)].iloc[0]
            if next_road['geom'].length < 3:
                next_neighbours = find_neighbouring_features(network_graph,
                                                        next_road['v' if nodeid == next_road['u'] else 'u'],
                                                        df=carriageways,
                                                        uuid_col='road_uuid',
                                                        exclude=[f['road_uuid'], next_road['road_uuid']])
                neighbours = pd.concat([neighbours, next_neighbours])
    
    next_uuid = f['u_next' if f['u'] == nodeid else 'v_next']
    for road_uuid, road in neighbours.groupby('road_uuid'):
        if len(road) == 1:
            if not np.isnan(next_uuid) and is_merge_with_next(f, next_uuid):
                cway = road.iloc[0]
                start_p = geom.intersection(cway['trimmed'])
                
                if start_p.type == "LineString":
                    continue
                
                end_p = Point(f['axis_geom'].geoms[0].coords[0 if f['u'] == nodeid else -1])
                
                if start_p == end_p:
                    continue
                
                dist = cway['trimmed'].project(start_p)
                
                if dist > 0 and dist < cway['trimmed'].length:                
                    # Connect the last part of the neighbour to this road
                    # We assume a single carriageway here
                    other_line = linemerge(cway['trimmed'])
                    if (cway['u'] == nodeid):
                        other_line = substring(other_line, 0, dist)
                    else:
                        other_line = substring(other_line, dist, other_line.length)

                    if (cway['u'] == nodeid) != ((f['u'] == nodeid) == (f['side'] == 1)):
                        other_line = LineString(list(other_line.coords[::-1]))

                    if (f['u'] == nodeid) == (f['side'] == 1):
                        geom = MultiLineString([other_line] + list(geom.geoms))
                    else:
                        geom = MultiLineString(list(geom.geoms) + [other_line])

                    return geom

                    new_lines = new_lines + [LineString((start_p, end_p))]
                
            continue
        
        for uuid, cway in road.set_index('uuid').iterrows():
            if cway['road_uuid'] != f['u_next'] and cway['road_uuid'] != f['v_next'] and f['trimmed'].intersects(cway['trimmed']):
                side = int(not cway['side'])
                
                other_side = road[road['side'] == side].iloc[0]
                
                start_p = geom.intersection(cway['trimmed'])
                
                if start_p.type == "LineString":
                    continue
                
                if f['dual']:
                    from_start = (f['u'] == nodeid) == (f['side'] == 1)
                else:
                    from_start = f['u'] == nodeid
                
                # That's probably a line crossing the same line in both sides
                if start_p.type == "MultiPoint":
                    start_p = linemerge(f['trimmed']).boundary[0 if from_start else 1]
                
                if not np.isnan(next_uuid) and is_merge_with_next(f, next_uuid):
                    end_p = Point(f['axis_geom'].geoms[0].coords[0 if f['u'] == nodeid else -1])
                else:  
                    # We'll use the "geom" because it's more reliable regarding the order of boundaries
                    if (other_side['u'] == nodeid) == (side == 1):
                        end_p = other_side['geom'].boundary[0]
                    else:
                        end_p = other_side['geom'].boundary[1]
                
                if end_p in geom.boundary:
                    continue
                
                if from_start:
                    end_p, start_p = (start_p, end_p)
                    
                if start_p == end_p:
                    continue

                new_lines = new_lines + [LineString((start_p, end_p))]
    
    geom = MultiLineString(new_lines)
    
    return geom

def next_too_short(f, nodeid):
    next_road = carriageways[carriageways['road_uuid'] == int(f['u_next' if f['u'] == nodeid else 'v_next'])].iloc[0]
    if next_road['geom'].length < 3:
        return np.isnan(next_road["v_next" if next_road['u'] == nodeid else "u_next"])
    
    return False

def is_merge_with_next(f, next_uuid):
    return f['dual'] == True and get_feature(carriageways, 'road_uuid', next_uuid)['dual'] == False

def extend_feature(f):
    geom = f['trimmed']
    
    if np.isnan(f['u_next']) or next_too_short(f, f['u']) or is_merge_with_next(f, f['u_next']):
        geom = extend_at_intersection(geom, f, f['u'])
    if np.isnan(f['v_next']) or next_too_short(f, f['v']) or is_merge_with_next(f, f['v_next']):
        geom = extend_at_intersection(geom, f, f['v'])
    
    return geom

carriageways['extended'] = carriageways.apply(extend_feature, axis=1)

geopandas.GeoDataFrame(carriageways.drop(['axis_geom', 'geometry_right', 'geom', 'trimmed'], 1),
                       geometry="extended",
                       crs="EPSG:28992").to_file('output/carriageways_extended.geojson', driver='GeoJSON')

f = carriageways[carriageways['uuid'] == 3813648913].iloc[0]
extend_feature(f).wkt

'MULTILINESTRING ((112669.6332406419 399839.2936344197, 112667.8998447234 399837.8775324319, 112673.7881345111 399811.7719808819, 112675.961589087 399811.2370963556))'

### Fix intersections when dual carriageways merge to one

When a carriageway ends up in an intersection and merges to one, then we should try to "snap" it to the perpendicular roads (if any).

The main idea is that we get the second-to-last point of the carriage way (that is, the point of the base of the small "triangle") and we compute the projection to all neighbours of the intersection. We should pick one of them based on a rule (e.g. the one with the closest distance?) and then make a new segment there.

In [847]:
from shapely.ops import substring, split

def move_vertex(line, orig_c, dest_c):
    """Will move the vertex orig_c to the position of dest_c.
    
    line --- A LineString of MultiLineString to process
    orig_c --- The original coords
    dest_c --- The destination coords
    """
    
    if line.type == "LineString":
        return LineString([dest_c if c==orig_c else c for c in line.coords])
    else:
        return MultiLineString([move_vertex(l, orig_c, dest_c) for l in line.geoms])

def cut_line(line, p, reverse = False):
    """Cuts the (Multi)LineString at the given point p
    
    line    --- A LineString or a MultiLineString to process
    p       --- The point where to cut the line string
    reverse --- Cut from the beginning or the end
    """
    
    if line.type == "LineString":
        dist = line.project(p)
        if reverse:
            return substring(line, 0, dist)
        else:
            return substring(line, dist, line.length)
    
    # Case of MultiLineString
    one_line = linemerge(line)
    
    assert one_line.type == "LineString"
    
    geoms = []
    for l in line.geoms:
        if l.project(p) > 0 and l.project(p) < l.length:
            geoms.append(cut_line(l, p, reverse))
        else:
            dist = one_line.project(p)
            dist_start = one_line.project(l.boundary[0])
            dist_end = one_line.project(l.boundary[1])
            
            if not reverse and dist_start <= dist and dist_end <= dist:
                continue
            elif reverse and dist_start >= dist and dist_end >= dist:
                continue
            geoms.append(l)
    
    return MultiLineString(geoms)

def fix_precision(geom, decimals=3):
    if geom.type == "LineString":
        return LineString([np.round(p, decimals) ])

def fix_merged_at_intersection(geom, f, nodeid):
    neighbours = find_neighbouring_features(network_graph,
                                            nodeid,
                                            df=carriageways,
                                            uuid_col='road_uuid',
                                            exclude=f['road_uuid'])
    
    if len(neighbours.groupby('road_uuid')) < 2:
        return geom
    
    # Here there is a MultiLineString so we need to pick the right one
    # and select the second-to-last point.    
    one_line = linemerge(geom)
        
    assert one_line.type == "LineString"
    
    p = Point(one_line.coords[1 if (f['u'] == nodeid) == (f['side'] == 1) else -2])
    end_p = Point(one_line.coords[0 if (f['u'] == nodeid) == (f['side'] == 1) else -1])
    
    az = get_line_azimuth(f['axis_geom'].geoms[0], f['u'] != nodeid)
    
    for uuid, n in neighbours.set_index('uuid').iterrows():
        if n['dual'] == True:
            continue

        cross_p = n["extended"].intersection(geom)
        if cross_p.type == "LineString":
            continue
                
        n_az = get_line_azimuth(n['axis_geom'].geoms[0], n['u'] != nodeid)
        
        angle = abs(n_az - az)
        
        if (angle < 70) and cross_p in one_line.boundary:
            continue
                
        if angle < 120 or (angle > 240 and angle < 300):
            l = 0
            m = n
            while l < LineString([p, end_p]).length:
                l = l + m['trimmed'].length
                dist = m['trimmed'].project(p)
                if dist > 0 and dist < m['trimmed'].length:
                    proj_p = m['trimmed'].interpolate(dist)
                    
                    # We snap the "remaining" part of the road to the neighbour
                    geom = move_vertex(geom, p.coords[0], proj_p.coords[0])
                    geom = cut_line(geom, proj_p, (f['u'] == nodeid) == (f['side'] == 0))
                                        
                    # Connect the last part of the neighbour to this road
                    # We assume a single carriageway here
                    other_line = linemerge(m['trimmed'])
                    if (n['u'] == nodeid):
                        other_line = substring(other_line, 0, dist)
                    else:
                        other_line = substring(other_line, dist, other_line.length)
                    
                    if (n['u'] == nodeid) != ((f['u'] == nodeid) == (f['side'] == 1)):
                        other_line = LineString(list(other_line.coords[::-1]))
                    
                    if (f['u'] == nodeid) == (f['side'] == 1):
                        geom = MultiLineString([other_line] + list(geom.geoms))
                    else:
                        geom = MultiLineString(list(geom.geoms) + [other_line])
                
                next_uuid = m['v_next' if m['u'] == nodeid else 'u_next']
                if np.isnan(next_uuid):
                    break
                else:
                    m = get_feature(carriageways, 'road_uuid', next_uuid)
    
    return geom        

def fix_merged_feature(f):
    """Returns a carriageway 'snapped' to the closest line."""
    
    geom = f['extended']
        
    if f["dual"] == False:
        return geom
    
    lines = carriageways[carriageways['road_uuid'] == f['road_uuid']]
        
    if is_dual_merge(lines, f['u']):
        geom = fix_merged_at_intersection(geom, f, f['u'])

    if is_dual_merge(lines, f['v']):
        geom = fix_merged_at_intersection(geom, f, f['v'])
    
    return geom
    
carriageways['snapped'] = carriageways.apply(fix_merged_feature, axis=1)

geopandas.GeoDataFrame(carriageways.drop(['axis_geom', 'geometry_right', 'geom', 'trimmed', 'extended'], 1),
                       geometry="snapped",
                       crs="EPSG:28992").to_file('output/carriageways_snapped.geojson', driver='GeoJSON')

# 3659445203
# 3596062575
# 3842874029
# f = carriageways[carriageways['uuid'] == 633889376].iloc[0]
# fix_merged_feature(f)

#### TODO

- In the current "snapped" pass we need make a last segment that takes the shape of the road that we "snap" to, instead of just making a simple line.
- We need to add a last pass where we intersect all lines of an intersection between them so that we find linear intersection in order to add a node in the lines that don't have one.

## Several notes and TODOS

1. **(DONE)** Smooth out the carriageways between them:
    - Normal case (two double-carriageways meet), we compute the mid point of the endpoints and join the lines.
    - When a double-carriageway meets a single one, we join with a triangle.
2. Find outliers that are too wide and are a dead-end.
3. Compute the new nodes of the network.

# Introduce flyovers

- What about the width calculation of flyovers? Is there an attribute in BGT about it? Can we compute the width by "isolating" the network and flyover polygon?

# Validation and evaluation

- Make sure we don't have those pedestrian area that are actually roads, but they are inconveniently classified in BGT as `voetpad`.
- We need to rectify our conditions for outliers regarding our decision of carriageway number from width.
- **INVESTIGATION REQUIRED**: If <1m segment is dual carriageway and neighbouring to both a single and a dual carriageway then we should make sure the methodology works.
- Ensure completeless of network (run a simple routing problem)
- Compute decorations like traffic lights
- Potentionally compute more interesting algorithms, e.g. what the best route about de-icing given the width of street (exploiting the semantic surfaces here).